## A. Configuraciones Generales.

In [ ]:
#1. Librerías.
%run "./librerias.ipynb"

In [ ]:
#2. Constantes.
#i. Generales.
%run "./constantes.ipynb"

#ii. Particulares.
#i. Dataset post Future Engineering del modelo.
dataset_input = dataset_file_fe_12_3
#ii. Mes de Test.
mes_test= mes_test
#iii. Ruta del modelo guardado post-optimización de hiperparámetros.
modelo =  "{}".format(modelos_path) + 'rf_1_3.sav'
#iv. Dataset donde exportar los resultados obtenidos.
dataset_output = path_output_m4_test

In [ ]:
#3. Funciones.
%run "./funciones.ipynb"

In [ ]:
#4. Lectura de dataframes post-feature engineering (.csv o .parquet), y modelos ya optimizados previamente (.txt).
data = pd.read_parquet(dataset_input)

In [ ]:
#5. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)
#ii. Elimino columnas de último momento por Data Concept.
columnas_de_interes_prestamos = data.filter(like='prestamos_personales').columns
data.drop(columnas_de_interes_prestamos,axis=1,inplace=True)

In [ ]:
#6. Dividimos entre conjuntos de datos.
#i. Datos a predecir.
X = data[data['foto_mes'] == mes_test]
#ii. Borramos las columnas de no interés.
X = X.drop(columns=['clase_ternaria'])

In [ ]:
#8. Obtenemos los hiperparámetros del modelo.
#i. Lo importo.
model_rf = pickle.load(open(modelo, 'rb'))
#ii. Extraigo los mejores parámetros del modelo.
best_params = model_rf.get_params()

## B. Predicción de Test.

In [ ]:
#i. Predecimos propiamente dicho.
predicciones = model_rf.predict_proba(X)[:, 1]

In [ ]:
#ii. Le pegamos la probabilidad de ser "BAJA" a cada cliente.
X['probabilidad'] = predicciones

In [ ]:
#iii. Nos quedamos solamente con las columnas de interés.
stacking_predictions = X[['numero_de_cliente', 'foto_mes', 'probabilidad']]

In [ ]:
#iv. Renombramos.
stacking_predictions.rename({"probabilidad":"probabilidad_modelo4"},axis=1,inplace=True)

In [ ]:
#v. Exportamos.
stacking_predictions.to_parquet(dataset_output)